In [13]:
!pip install transformers
!pip install spotipy
!pip install lyricsgenius

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import torch

In [3]:
spotify_client_id = 'c7e26856bb5d4430b2681da841f04002'
spotify_client_secret = '23cb3f88565a477182d36880596820bb'

genius_access_token = 'u3oyNBP3IiXfHAksSzWEd-pkHot_lndqB6664h4Pf_tDKxv7IZvJD7ykS2z8Dnlk'


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=spotify_client_id,
                                                           client_secret=spotify_client_secret))

genius = lyricsgenius.Genius(genius_access_token)

In [10]:
def get_lyrics(artist_name, track_name):
    try:
        search_results = sp.search(q='track:"' + track_name + '" artist:"' + artist_name + '"', type='track')['tracks']['items']
        if search_results:
            track_id = search_results[0]['id']
            song = genius.search_song(track_name, artist_name)
            return song.lyrics if song else "Letra não encontrada"
        else:
            return "Letra não encontrada"
    except IndexError:
        return "Letra não encontrada"

In [15]:
model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
def classify_sentiment(text):
    inputs = tokenizer(lyrics, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    return predicted_class

In [17]:
while True:
    track_name = input("Digite o nome da música (ou 'exit' para sair): ")
    if track_name.lower() == 'exit':
        break
    artist_name = input("Digite o nome do artista: ")

    lyrics = get_lyrics(artist_name, track_name)

    if lyrics != "Letra não encontrada":
        sentiment_class = classify_sentiment(lyrics)
        sentiment_label = {
            0: 'Muito negativo',
            1: 'Negativo',
            2: 'Neutro',
            3: 'Positivo',
            4: 'Muito positivo'
        }
        sentiment = sentiment_label.get(sentiment_class, 'Indeterminado')
    else:
        sentiment = "Letra não encontrada"

    print(f'\nSentimento da música "{track_name}" de "{artist_name}": {sentiment}\n')
    if lyrics != "Letra não encontrada":
        print("\nLetra da música:\n")
        print(lyrics)

Digite o nome da música (ou 'exit' para sair): Live Forever
Digite o nome do artista: Oasis
Searching for "Live Forever" by Oasis...
Done.

Sentimento da música "Live Forever" de "Oasis": Muito positivo


Letra da música:

45 ContributorsLive Forever Lyrics[Intro]
Oh yeah

[Verse 1]
Maybe I don't really wanna know
How your garden grows
'Cause I just wanna fly
Lately, did you ever feel the pain
In the morning rain
As it soaks you to the bone?

[Chorus 1]
Maybe I just wanna fly
Wanna live, I don't wanna die
Maybe I just wanna breathe
Maybe I just don't believe
Maybe you're the same as me
We see things they'll never see
You and I are gonna live forever

[Verse 2]
I said maybe I don't really wanna know
How your garden grows
'Cause I just wanna fly
Lately, did you ever feel the pain
In the morning rain
As it soaks you to the bone?
You might also like[Chorus 2]
Maybe I will never be
All the things that I wanna be
Now is not the time to cry
Now's the time to find out why
I think you're the sa